# begin

# 1

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import interp1d

# --- 1. 加载参数（来自YAML配置文件） ---
PARAMS = {
    'Q_intake_max': 0.6,          # 最大取水量 (m³/s)
    'Storage_max': 25000,         # 储水罐最大容量 (m³)
    'Cost_shutdown_hourly': 80000, # 单位小时停泵成本 (元/小时)
    'Penalty_shortage': 5000,     # 缺水惩罚系数 (元/m³，表征供水可靠性价值)
    'Penalty_pollution': 1e9,     # 污染惩罚系数 (元/kg，表征健康风险成本)
    'Initial_Storage_Pct': 0.8    # 初始储水率 (假设储水罐初始为80%满水状态)
}

# --- 2. 加载数据 ---
# 加载提供的污染物浓度数据
df_conc = pd.read_csv('x=30km_C(x,t).csv')

# 加载用水需求曲线（解析文本格式数据）
demand_data = """hour_of_day,relative_demand
0,0.8253
1,0.8061
2,0.8002
3,0.8078
4,0.8286
5,0.861
6,0.9029
7,0.9515
8,1.0033
9,1.0549
10,1.1028
11,1.1436
12,1.1747
13,1.1939
14,1.1998
15,1.1922
16,1.1714
17,1.139
18,1.0971
19,1.0485
20,0.9967
21,0.9451
22,0.8972
23,0.8564"""
from io import StringIO
df_demand = pd.read_csv(StringIO(demand_data))

# --- 3. 数据预处理 ---
# 构建用水需求插值函数
avg_demand_m3s = 0.6 * 0.9 # 假设平均用水量为最大取水量的90%，预留补水空间
func_demand = interp1d(df_demand['hour_of_day'], 
                       df_demand['relative_demand'] * avg_demand_m3s, 
                       kind='cubic', fill_value="extrapolate")

def get_demand_m3s(time_hour):
    """根据当前时刻计算瞬时用水量 (m³/s)"""
    hour_of_day = time_hour % 24
    return float(func_demand(hour_of_day))

# 生成D10和D200情景的模拟数据以进行鲁棒性分析（原始CSV仅提供D50数据）
# D10情景：峰值浓度更高，污染持续时间更短
# D200情景：峰值浓度更低，污染持续时间更长
t = df_conc['Time_hours'].values
c_base = df_conc['Concentration_D50_mgL'].values
peak_idx = np.argmax(c_base)
peak_time = t[peak_idx]

# 模拟数据生成（若有实际模拟数据，可替换此部分代码）
df_conc['Concentration_D10_mgL'] = c_base * 1.5 * np.exp(-0.5 * ((t - peak_time)/5)**2) # 模拟高浓度窄峰情景
df_conc['Concentration_D200_mgL'] = np.convolve(c_base, np.ones(50)/50, mode='same') # 模拟低浓度宽峰情景

print("数据加载完成。")
print(f"时间序列长度: {len(df_conc)} 个数据点")
print(f"D50情景最大污染物浓度: {df_conc['Concentration_D50_mgL'].max():.4f} mg/L")

数据加载完成。
时间序列长度: 5237 个数据点
D50情景最大污染物浓度: 0.4702 mg/L


# V2 2

In [8]:
# --- 更新参数权重 ---
# 加大污染惩罚，确保"安全第一"
PARAMS['Penalty_pollution'] = 1e7  # 从 1e6 提升到 1e7 (1000万/kg)
PARAMS['Mandatory_Threshold'] = 0.3 # 题目给定的红线

def run_operation_simulation_v2(c_series, time_series, c_off, c_on, params):
    dt_hours = time_series[1] - time_series[0]
    dt_sec = dt_hours * 3600
    
    # 状态变量
    storage = params['Storage_max'] * params['Initial_Storage_Pct']
    is_intake_open = True
    
    # 统计变量
    total_shutdown_hours = 0
    total_shortage_m3 = 0
    total_pollutant_kg = 0
    illegal_intake_hours = 0 # 记录违规时长
    
    storage_history = []
    status_history = []
    
    for i, c_river in enumerate(c_series):
        t_curr = time_series[i]
        
        # 1. 运营逻辑 (带滞回)
        # 即使优化器想开，如果超过强制阈值 0.3，物理上也必须认为这是不可接受的风险
        # 但为了让优化器自己“学会”这个规则，我们把逻辑保持原样，
        # 在后面计算成本时对违规行为进行重罚。
        if is_intake_open:
            if c_river > c_off:
                is_intake_open = False
        else:
            if c_river < c_on:
                is_intake_open = True
                
        # 2. 水量平衡
        demand_m3s = get_demand_m3s(t_curr)
        
        # 进水逻辑
        potential_intake = 0
        if is_intake_open:
            potential_intake = params['Q_intake_max']
            
            # 计算吸入的污染物
            pollutant_rate_kgs = potential_intake * c_river / 1000
            total_pollutant_kg += pollutant_rate_kgs * dt_sec
            
            # 【关键修正】检测非法取水
            # 如果当前河水浓度 > 0.3 且还在取水，这是严重违规
            if c_river > params['Mandatory_Threshold']:
                illegal_intake_hours += dt_hours

        # 储水更新 (dS/dt = In - Out)
        net_flow = potential_intake - demand_m3s
        storage += net_flow * dt_sec
        
        # 3. 边界约束
        if storage > params['Storage_max']:
            storage = params['Storage_max']
            
        if storage < 0:
            shortage = abs(storage) 
            total_shortage_m3 += shortage
            storage = 0 
            
        # 4. 记录状态
        if not is_intake_open:
            total_shutdown_hours += dt_hours
            
        storage_history.append(storage)
        status_history.append(1 if is_intake_open else 0)
    
    # --- 成本计算 ---
    cost_shutdown = total_shutdown_hours * params['Cost_shutdown_hourly']
    cost_reliability = total_shortage_m3 * params['Penalty_shortage']
    cost_safety = total_pollutant_kg * params['Penalty_pollution']
    
    # 【新增】非法操作惩罚 (惩罚系数极大，迫使优化器避开 > 0.3 的区域)
    cost_illegal = illegal_intake_hours * 1e9 
    
    total_cost = cost_shutdown + cost_reliability + cost_safety + cost_illegal
    
    return {
        'Total_Cost': total_cost,
        'Shutdown_Hours': total_shutdown_hours,
        'Shortage_m3': total_shortage_m3,
        'Pollutant_kg': total_pollutant_kg,
        'Illegal_Hours': illegal_intake_hours,
        'Storage_History': storage_history,
        'Status_History': status_history,
        'Costs': (cost_shutdown, cost_reliability, cost_safety, cost_illegal)
    }

print("模拟引擎 V2 已更新：加入了强制关停惩罚机制。")

模拟引擎 V2 已更新：加入了强制关停惩罚机制。


## 2

In [2]:
def run_operation_simulation(c_series, time_series, c_off, c_on, params):
    """
    基于指定控制规则，模拟供水系统运行过程
    参数:
        c_series: 河流污染物浓度时间序列
        time_series: 时间序列
        c_off: 停泵阈值
        c_on: 启泵阈值
        params: 系统参数字典
    返回:
        包含各项运行指标的字典
    """
    dt_hours = time_series[1] - time_series[0]  # 时间步长（小时）
    dt_sec = dt_hours * 3600                    # 时间步长（秒）
    
    # 系统状态变量初始化
    storage = params['Storage_max'] * params['Initial_Storage_Pct']  # 初始储水量
    is_intake_open = True  # 取水口初始状态为开启
    
    # 运行指标跟踪变量
    total_shutdown_hours = 0  # 累计停泵时长
    total_shortage_m3 = 0     # 累计缺水量
    total_pollutant_kg = 0    # 累计污染物摄入量
    
    storage_history = []  # 储水量变化历史
    status_history = []   # 取水口状态变化历史
    
    for i, c_river in enumerate(c_series):
        t_curr = time_series[i]
        
        # 1. 滞回控制逻辑
        if is_intake_open:
            if c_river > c_off:
                is_intake_open = False  # 浓度超过停泵阈值，关闭取水口
        else:
            if c_river < c_on:
                is_intake_open = True   # 浓度低于启泵阈值，开启取水口
                
        # 2. 水量平衡计算
        demand_m3s = get_demand_m3s(t_curr)  # 当前时刻用水量
        
        # 计算取水量
        if is_intake_open:
            # 取水策略：在最大取水量限制内，优先满足用水需求并补充储水
            # 简化模型：除非储水罐满水，否则按最大取水量取水
            potential_intake = params['Q_intake_max']
            # 计算污染物摄入速率
            # 单位换算：(m³/s * mg/L) = (m³/s * g/m³) = g/s → 除以1000得到 kg/s
            pollutant_rate_kgs = potential_intake * c_river / 1000
            total_pollutant_kg += pollutant_rate_kgs * dt_sec
        else:
            potential_intake = 0  # 停泵状态下取水量为0
            
        # 更新储水量
        # 水量平衡方程：dS/dt = 取水量 - 用水量
        net_flow = potential_intake - demand_m3s
        storage += net_flow * dt_sec
        
        # 3. 约束条件检查
        # 储水罐溢水处理
        if storage > params['Storage_max']:
            storage = params['Storage_max']
            
        # 缺水情况处理
        if storage < 0:
            shortage = abs(storage)  # 实际缺水量
            total_shortage_m3 += shortage
            storage = 0  # 储水罐排空
            
        # 4. 运行指标更新
        if not is_intake_open:
            total_shutdown_hours += dt_hours
            
        storage_history.append(storage)
        status_history.append(1 if is_intake_open else 0)
        
    # 计算目标函数值（总损失）
    cost_shutdown = total_shutdown_hours * params['Cost_shutdown_hourly']
    cost_reliability = total_shortage_m3 * params['Penalty_shortage']
    cost_safety = total_pollutant_kg * params['Penalty_pollution']
    
    total_cost = cost_shutdown + cost_reliability + cost_safety
    
    return {
        'Total_Cost': total_cost,
        'Shutdown_Hours': total_shutdown_hours,
        'Shortage_m3': total_shortage_m3,
        'Pollutant_kg': total_pollutant_kg,
        'Storage_History': storage_history,
        'Status_History': status_history,
        'Costs': (cost_shutdown, cost_reliability, cost_safety)
    }

# --- 测试运行 ---
test_res = run_operation_simulation(
    df_conc['Concentration_D50_mgL'].values, 
    df_conc['Time_hours'].values, 
    c_off=0.1, c_on=0.05, params=PARAMS
)
print("测试运行结果 (停泵阈值=0.1 mg/L, 启泵阈值=0.05 mg/L):")
print(f"总损失: {test_res['Total_Cost']:,.0f} 元")
print(f"累计停泵时长: {test_res['Shutdown_Hours']:.2f} 小时")
print(f"累计缺水量: {test_res['Shortage_m3']:.2f} m³")
print(f"累计污染物摄入量: {test_res['Pollutant_kg']:.4f} kg")

测试运行结果 (停泵阈值=0.1 mg/L, 启泵阈值=0.05 mg/L):
总损失: 525,237,462 元
累计停泵时长: 65.09 小时
累计缺水量: 103691.65 m³
累计污染物摄入量: 1.5718 kg


# V2 3

In [9]:
# Grid Definition - 修正范围，不超过0.3
c_off_values = np.linspace(0.01, 0.30, 30) 
results_grid = []

print("重新开始网格搜索优化 (V2)...")

for c_off in c_off_values:
    # 启泵阈值必须小于等于停泵阈值
    # 我们测试几个典型的回差 (Hysteresis) 比例
    for ratio in [0.5, 0.7, 0.9, 0.95]: 
        c_on = c_off * ratio
        
        res = run_operation_simulation_v2(
            df_conc['Concentration_D50_mgL'].values, 
            df_conc['Time_hours'].values, 
            c_off, c_on, PARAMS
        )
        
        results_grid.append({
            'C_off': c_off,
            'C_on': c_on,
            'Total_Cost': res['Total_Cost'],
            'Shutdown_Cost': res['Costs'][0],
            'Reliability_Cost': res['Costs'][1],
            'Safety_Cost': res['Costs'][2],
            'Illegal_Penalty': res['Costs'][3],
            'Shutdown_Hours': res['Shutdown_Hours'],
            'Shortage_m3': res['Shortage_m3']
        })

df_opt = pd.DataFrame(results_grid)
best_strategy = df_opt.loc[df_opt['Total_Cost'].idxmin()]

print("\n=== 修正后的最优策略 (基准 D50) ===")
print(f"停泵阈值 C_off: {best_strategy['C_off']:.3f} mg/L")
print(f"启泵阈值 C_on:  {best_strategy['C_on']:.3f} mg/L")
print(f"最小总损失: {best_strategy['Total_Cost']:,.0f} 元")
print("-" * 30)
print(f"  - 停泵时长:      {best_strategy['Shutdown_Hours']:.2f} 小时")
print(f"  - 停泵成本:      {best_strategy['Shutdown_Cost']:,.0f} 元")
print(f"  - 缺水量:        {best_strategy['Shortage_m3']:,.0f} m3")
print(f"  - 供水可靠性成本: {best_strategy['Reliability_Cost']:,.0f} 元")
print(f"  - 水质安全成本:   {best_strategy['Safety_Cost']:,.0f} 元")
print(f"  - 违规惩罚:       {best_strategy['Illegal_Penalty']:,.0f} 元")

if best_strategy['Shortage_m3'] > 0:
    print("\n【重要发现】: 即使在最优策略下，仍出现了供水短缺。")
    print("这证明当前的 25,000 m3 储水量不足以应对此次持续近 100 小时的污染事件。")
    print("这是写进 Memo 的关键点：必须调配外部水源（如水车）或扩建储水池。")

重新开始网格搜索优化 (V2)...

=== 修正后的最优策略 (基准 D50) ===
停泵阈值 C_off: 0.300 mg/L
启泵阈值 C_on:  0.150 mg/L
最小总损失: 439,363,807 元
------------------------------
  - 停泵时长:      49.47 小时
  - 停泵成本:      3,957,800 元
  - 缺水量:        70,741 m3
  - 供水可靠性成本: 353,703,671 元
  - 水质安全成本:   81,702,336 元
  - 违规惩罚:       0 元

【重要发现】: 即使在最优策略下，仍出现了供水短缺。
这证明当前的 25,000 m3 储水量不足以应对此次持续近 100 小时的污染事件。
这是写进 Memo 的关键点：必须调配外部水源（如水车）或扩建储水池。


## 3

In [4]:
# 定义网格搜索参数范围
c_off_values = np.linspace(0.01, 0.4, 20)  # 停泵阈值测试范围
results_grid = []

print("开始执行网格搜索优化...")

for c_off in c_off_values:
    # 启泵阈值需小于等于停泵阈值，测试不同比例系数
    for ratio in [0.1, 0.5, 0.9, 1.0]: 
        c_on = c_off * ratio
        
        res = run_operation_simulation(
            df_conc['Concentration_D50_mgL'].values, 
            df_conc['Time_hours'].values, 
            c_off, c_on, PARAMS
        )
        
        results_grid.append({
            'C_off': c_off,
            'C_on': c_on,
            'Total_Cost': res['Total_Cost'],
            'Shutdown_Cost': res['Costs'][0],
            'Reliability_Cost': res['Costs'][1],
            'Safety_Cost': res['Costs'][2],
            'Shutdown_Hours': res['Shutdown_Hours']
        })

# 整理优化结果
df_opt = pd.DataFrame(results_grid)
best_strategy = df_opt.loc[df_opt['Total_Cost'].idxmin()]

print("\n=== 基准情景(D50)最优控制策略 ===")
print(f"停泵阈值 $C_{{off}}$: {best_strategy['C_off']:.3f} mg/L")
print(f"启泵阈值 $C_{{on}}$:  {best_strategy['C_on']:.3f} mg/L")
print(f"最小总损失: {best_strategy['Total_Cost']:,.0f} 元")
print(f"  - 停泵成本:  {best_strategy['Shutdown_Cost']:,.0f} 元")
print(f"  - 供水可靠性成本: {best_strategy['Reliability_Cost']:,.0f} 元")
print(f"  - 水质安全成本:      {best_strategy['Safety_Cost']:,.0f} 元")

开始执行网格搜索优化...

=== 基准情景(D50)最优控制策略 ===
停泵阈值 $C_{off}$: 0.400 mg/L
启泵阈值 $C_{on}$:  0.400 mg/L
最小总损失: 208,567,885 元
  - 停泵成本:  2,464,000 元
  - 供水可靠性成本: 183,550,617 元
  - 水质安全成本:      22,553,268 元


# V2 4

In [ ]:
# 可视化最优结果
best_run = run_operation_simulation_v2(
    df_conc['Concentration_D50_mgL'].values, 
    df_conc['Time_hours'].values, 
    best_strategy['C_off'], best_strategy['C_on'], PARAMS
)

plt.figure(figsize=(12, 8))

# 子图1：浓度与操作
plt.subplot(2, 1, 1)
plt.plot(df_conc['Time_hours'], df_conc['Concentration_D50_mgL'], 'k-', alpha=0.6, label='River Concentration')
plt.axhline(best_strategy['C_off'], color='r', linestyle='--', label=f'Shutdown Threshold ({best_strategy["C_off"]:.2f})')
plt.axhline(best_strategy['C_on'], color='g', linestyle=':', label=f'Restart Threshold ({best_strategy["C_on"]:.2f})')
plt.axhline(0.3, color='purple', linewidth=2, label='Mandatory Limit (0.3)')

# 填充关停区域
is_closed = [status == 0 for status in best_run['Status_History']]
plt.fill_between(df_conc['Time_hours'], 0, 0.5, where=is_closed, color='red', alpha=0.1, label='Intake Closed')

plt.title('Optimal Shutdown Strategy', fontsize=14, fontweight='bold')
plt.ylabel('Concentration (mg/L)')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

# 子图2：储水量变化
plt.subplot(2, 1, 2)
storage_vol = np.array(best_run['Storage_History'])
plt.plot(df_conc['Time_hours'], storage_vol, 'b-', linewidth=2, label='Tank Storage')
plt.axhline(PARAMS['Storage_max'], color='gray', linestyle='--', label='Max Capacity')
plt.axhline(0, color='red', linewidth=2, label='Empty (Failure)')

# 标记缺水区域
shortage_periods = storage_vol <= 0
plt.fill_between(df_conc['Time_hours'], 0, PARAMS['Storage_max'], where=shortage_periods, color='orange', alpha=0.3, label='Water Shortage Period')

plt.title('Emergency Storage Dynamics', fontsize=14, fontweight='bold')
plt.xlabel('Time (Hours)')
plt.ylabel('Storage Volume (m3)')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4

In [ ]:
# --- 1. 鲁棒性验证 ---
scenarios = ['Concentration_D10_mgL', 'Concentration_D50_mgL', 'Concentration_D200_mgL']
robustness_results = []

# 提取最优阈值
opt_c_off = best_strategy['C_off']
opt_c_on = best_strategy['C_on']

for scen in scenarios:
    res = run_operation_simulation(
        df_conc[scen].values, 
        df_conc['Time_hours'].values, 
        opt_c_off, opt_c_on, PARAMS
    )
    robustness_results.append({
        'Scenario': scen.replace('Concentration_', '').replace('_mgL', ''),
        'Total_Cost': res['Total_Cost'],
        'Shutdown_Hours': res['Shutdown_Hours'],
        'Shortage_m3': res['Shortage_m3'],
        'Pollutant_kg': res['Pollutant_kg']
    })

df_robust = pd.DataFrame(robustness_results)

# --- 2. 结果可视化 ---
plt.figure(figsize=(14, 10))
plt.style.use('seaborn-v0_8-whitegrid')

# 子图1：优化结果图谱
plt.subplot(2, 2, 1)
sc = plt.scatter(df_opt['C_off'], df_opt['C_on'], c=np.log10(df_opt['Total_Cost']), 
                 cmap='viridis', s=50, edgecolors='k')
plt.colorbar(sc, label='总损失对数 (Log10)')
plt.scatter(best_strategy['C_off'], best_strategy['C_on'], color='red', marker='*', s=200, label='最优阈值')
plt.title('优化图谱：损失与控制阈值关系', fontsize=12, fontweight='bold')
plt.xlabel('停泵阈值 $C_{off}$ (mg/L)')
plt.ylabel('启泵阈值 $C_{on}$ (mg/L)')
plt.legend()

# 子图2：最优策略运行时序图
# 重新运行最优策略以获取时序数据
best_run = run_operation_simulation(df_conc['Concentration_D50_mgL'].values, 
                                    df_conc['Time_hours'].values, 
                                    opt_c_off, opt_c_on, PARAMS)

ax2 = plt.subplot(2, 2, 2)
ax2.plot(df_conc['Time_hours'], df_conc['Concentration_D50_mgL'], 'k-', label='河流污染物浓度', alpha=0.6)
ax2.axhline(opt_c_off, color='r', linestyle='--', label=f'停泵阈值: {opt_c_off:.2f} mg/L')
ax2.axhline(opt_c_on, color='g', linestyle='--', label=f'启泵阈值: {opt_c_on:.2f} mg/L')
ax2.set_xlabel('时间 (小时)')
ax2.set_ylabel('污染物浓度 (mg/L)')
ax2.set_title('最优控制策略执行过程', fontsize=12, fontweight='bold')
ax2.legend(loc='upper left')

# 叠加取水口状态阴影标注
ax2_twin = ax2.twinx()
ax2_twin.fill_between(df_conc['Time_hours'], 0, 1, 
                      where=[s==0 for s in best_run['Status_History']], 
                      color='red', alpha=0.2, label='取水口关闭')
ax2_twin.set_yticks([])
ax2_twin.legend(loc='upper right')

# 子图3：储水罐水位变化曲线
plt.subplot(2, 2, 3)
plt.plot(df_conc['Time_hours'], np.array(best_run['Storage_History'])/1000, 'b-', lw=2)
plt.axhline(PARAMS['Storage_max']/1000, color='gray', linestyle=':', label='最大储水量')
plt.axhline(0, color='red', lw=2, label='储水耗尽（供水失效）')
plt.title('应急储水罐水位动态变化', fontsize=12, fontweight='bold')
plt.xlabel('时间 (小时)')
plt.ylabel('储水量 ($1000 \ m^3$)')
plt.legend()

# 子图4：多情景鲁棒性对比
plt.subplot(2, 2, 4)
x = np.arange(len(df_robust))
width = 0.35
plt.bar(x - width/2, df_robust['Shutdown_Hours'], width, label='停泵时长', color='skyblue')
plt.bar(x + width/2, df_robust['Shortage_m3'], width, label='缺水量 ($m^3$)', color='salmon')
plt.xticks(x, df_robust['Scenario'])
plt.title('鲁棒性分析：最优策略在不同情景下的表现', fontsize=12, fontweight='bold')
plt.ylabel('数值')
plt.legend()

plt.tight_layout()
plt.show()

print("\n=== 鲁棒性分析报告 ===")
print(df_robust.to_string(index=False))

# 5

# end